# Pydantic
playing with pydantic model and settings features

In [ ]:
import pydantic

In [ ]:
import json

gps_json = '{"latitude": 34.7276107, "longitude": -122.0421016220921, "speed": 0.0, "heading": 2.0}'

gps_dict = json.loads(gps_json)
gps_list = list(gps_dict.values())

gps_latitude = gps_dict["latitude"]
gps_longitude = gps_dict["longitude"]
gps_speed = gps_dict["speed"]
gps_heading = gps_dict["heading"]

print(gps_dict)

## Dataclass
Pydantic provides a drop-in replacement for standard python dataclasses with additional validation functionality.

In [ ]:
import dataclasses

@pydantic.dataclasses.dataclass
class GPSData_pydantic_dataclass:
    latitude: float
    longitude: float
    speed: float
    heading: float

@dataclasses.dataclass
class GPSData_standard_dataclass:
    latitude: float
    longitude: float
    speed: float
    heading: float

In [ ]:
# valid ways of instantiating
# keyword
gps_pydantic_dataclass = GPSData_pydantic_dataclass(
    latitude=gps_latitude,
    longitude=gps_longitude,
    speed=gps_speed,
    heading=gps_heading,
)
# positional
gps_pydantic_dataclass1 = GPSData_pydantic_dataclass(gps_latitude, gps_longitude, gps_speed, gps_heading)
# keyword by unpacking dict
gps_pydantic_dataclass2 = GPSData_pydantic_dataclass(**gps_dict)
# positional by unpacking list/tuple
gps_pydantic_dataclass3 = GPSData_pydantic_dataclass(*gps_list)

# all create equivalent objects
assert gps_pydantic_dataclass == gps_pydantic_dataclass1 == gps_pydantic_dataclass2 == gps_pydantic_dataclass3

# same functionality as standard dataclass
# keyword
gps_standard_dataclass = GPSData_standard_dataclass(
    latitude=gps_latitude,
    longitude=gps_longitude,
    speed=gps_speed,
    heading=gps_heading,
)
# positional
gps_standard_dataclass1 = GPSData_standard_dataclass(gps_latitude, gps_longitude, gps_speed, gps_heading)
# keyword by unpacking dict
gps_standard_dataclass2 = GPSData_standard_dataclass(**gps_dict)
# positional by unpacking list/tuple
gps_standard_dataclass3 = GPSData_standard_dataclass(*gps_list)

# all create equivalent objects
assert gps_standard_dataclass == gps_standard_dataclass1 == gps_standard_dataclass2 == gps_standard_dataclass3

Note that these dataclasses cannot be compared to each other, because the pydantic dataclass adds additional attributes that leads to a different `__dict__` attribute (which can be gotten with `vars()`). They would need to explicitly compared by looking at `__dict__` from the standard dataclass since it should be a subset, using the hidden attribute `__dataclass_fields__`, or using standard `dataclasses.asdict()` function

In [ ]:
print(
    vars(gps_pydantic_dataclass),
    vars(gps_standard_dataclass),
    sep="\n",
)

In [ ]:
print(f"{(gps_pydantic_dataclass == gps_standard_dataclass)=}")
# get values directly by referencing __dataclass_fields__ attribute
assert (
    all(
        getattr(gps_pydantic_dataclass, k) == getattr(gps_standard_dataclass, k)
        for k in vars(gps_standard_dataclass)
        # for k in gps_pydantic_dataclass.__dataclass_fields__
    )
)
print(f"{(dataclasses.asdict(gps_pydantic_dataclass) == dataclasses.asdict(gps_standard_dataclass))=}")

In [ ]:
# str/repr
print(
    "GPSData_pydantic_dataclass:",
    f"  str:  {gps_pydantic_dataclass}",
    f"  repr: {gps_pydantic_dataclass!r}",
    "GPSData_standard_dataclass:",
    f"  str:  {gps_standard_dataclass}",
    f"  repr: {gps_standard_dataclass!r}",
    sep="\n",
)

In [ ]:
import inspect

# instance methods
print(
    "GPSData_pydantic_dataclass:",
    *[
        f"{field}: {value}"
        for field, value in inspect.getmembers(gps_pydantic_dataclass)
        if not field.startswith("_")
    ],
    sep="\n  ",
)
print(
    "GPSData_standard_dataclass:",
    *[
        f"{field}: {value}"
        for field, value in inspect.getmembers(gps_standard_dataclass)
        if not field.startswith("_")
    ],
    sep="\n  ",
)

In [ ]:
# validation just checks that the input data can be coerced to the expected type
gps_latitude_str = str(gps_latitude)
gps_longitude_padded_str = f"{gps_longitude}    "
gps_speed_invalid_str = "some string"

# invalid speed input
try:
    print(
        GPSData_pydantic_dataclass(
            latitude=gps_latitude_str,
            longitude=gps_longitude_padded_str,
            speed=gps_speed_invalid_str,
            heading=0,
        )
    )
except pydantic.ValidationError as e:
    print(e)

# valid, coercible inputs
try:
    print(
        GPSData_pydantic_dataclass(
            latitude=gps_latitude_str,
            longitude=gps_longitude_padded_str,
            speed=gps_speed,
            heading=0,
        )
    )
except pydantic.ValidationError as e:
    print(e)

# but with standard dataclasses, the values are not checked or coerced
print(
    GPSData_standard_dataclass(
        latitude=gps_latitude_str,
        longitude=gps_longitude_padded_str,
        speed=gps_speed_invalid_str,
        heading=0,
    )
)

## BaseModel
Similar to dataclasses, with additional functionality like (de)serializing from json

In [ ]:
class GPSData_pydantic_base_model(pydantic.BaseModel):
    latitude: float
    longitude: float
    speed: float
    heading: float

Note that when instantiating, positional arguments are not allowed so keywords must be used

In [ ]:
# valid ways of instantiating
# keyword
gps_pydantic_base_model = GPSData_pydantic_base_model(
    latitude=gps_latitude,
    longitude=gps_longitude,
    speed=gps_speed,
    heading=gps_heading,
)
# unpacking dict
gps_pydantic_base_model1 = GPSData_pydantic_base_model(**gps_dict)
# from dict using parse_obj
gps_pydantic_base_model2 = GPSData_pydantic_base_model.parse_obj(gps_dict)
# from json using parse_raw
gps_pydantic_base_model3 = GPSData_pydantic_base_model.parse_raw(gps_json)
# a json file could also potentially be used
# gps_pydantic_base_model = GPSData_pydantic_base_model.parse_file(gps_file.json)
# construct can be used to bypass validation
gps_pydantic_base_model4 = GPSData_pydantic_base_model.construct(**gps_dict)
# from an existing object using deep/shallow copy
gps_pydantic_base_model5 = GPSData_pydantic_base_model.copy(gps_pydantic_base_model)
gps_pydantic_base_model6 = GPSData_pydantic_base_model.copy(gps_pydantic_base_model, deep=True)

# all create equivalent objects
assert gps_pydantic_base_model == gps_pydantic_base_model1 == gps_pydantic_base_model2 == gps_pydantic_base_model3 == gps_pydantic_base_model4 == gps_pydantic_base_model5 == gps_pydantic_base_model6


In [ ]:
# str/repr, dict, json
print(
    "GPSData_pydantic_base_model:",
    f"str:  {gps_pydantic_base_model}",
    f"repr: {gps_pydantic_base_model!r}",
    f"dict: {gps_pydantic_base_model.dict()!r}",
    f"json: {gps_pydantic_base_model.json()!r}",
    sep="\n  ",
)

In [ ]:
import inspect

# instance/class methods
print(
    "GPSData_pydantic_base_model:",
    *[
        f"{field}: {value}"
        for field, value in inspect.getmembers(gps_pydantic_base_model)
        if not field.startswith("_")
    ],
    sep="\n  ",
)

## Validation and constraints
Validating values beyond type checking can be configured using [helper constrained types](https://pydantic-docs.helpmanual.io/usage/types/#constrained-types), the [`Field` class](https://pydantic-docs.helpmanual.io/usage/schema/#field-customization), or custom [validation functions](https://pydantic-docs.helpmanual.io/usage/validators/).

Note that constrained types can be used in conjunction with `Field`, but constraints specified using `Field` [will not be enforced](https://pydantic-docs.helpmanual.io/usage/schema/#unenforced-field-constraints) and an Exception is thrown if attempted

In [ ]:
class GPSData_helpers(pydantic.BaseModel):
    latitude: pydantic.confloat(ge=-90, le=90)
    longitude: pydantic.confloat(ge=-180, le=180)
    speed: pydantic.NonNegativeFloat
    heading: pydantic.confloat(ge=0, le=360)

class GPSData_fields(pydantic.BaseModel):
    latitude: float | None = pydantic.Field(ge=-90, le=90, default=None)
    longitude: float = pydantic.Field(ge=-180, le=180)
    speed: float = pydantic.Field(ge=0)
    heading: float = pydantic.Field(ge=0, le=360)
    class Config:
        validate_assignment = True

In [ ]:
gps_latitude_str = str(gps_latitude)
gps_longitude_padded_str = f"{gps_longitude}    "
gps_longitude_padded_out_str = f"{gps_longitude + 370}    "
gps_speed_invalid_str = "some string"

gps_data_invalid_speed = {
    "latitude": gps_latitude_str,
    "longitude": gps_longitude_padded_str,
    "speed": gps_speed_invalid_str,
    "heading": 0,
}
gps_data_in_constraint = {
    "latitude": gps_latitude_str,
    "longitude": gps_longitude_padded_str,
    "speed": gps_speed,
    "heading": 0,
}
gps_data_out_constraint = {
    "latitude": gps_latitude + 100,
    "longitude": gps_longitude_padded_out_str,
    "speed": gps_speed - 10,
    "heading": -10,
}

# invalid speed input
try:
    print(GPSData_pydantic_base_model.parse_obj(gps_data_invalid_speed))
except pydantic.ValidationError as e:
    print(e)

# valid, coercible inputs
print(
    GPSData_pydantic_base_model.parse_obj(gps_data_in_constraint),
    GPSData_helpers.parse_obj(gps_data_in_constraint),
    GPSData_fields.parse_obj(gps_data_in_constraint),
    sep="\n",
)

# coercible, but not within constraints
print(GPSData_pydantic_base_model.parse_obj(gps_data_out_constraint))
try:
    print(GPSData_helpers.parse_obj(gps_data_out_constraint))
except pydantic.ValidationError as e:
    print(e)
try:
    print(GPSData_fields.parse_obj(gps_data_out_constraint))
except pydantic.ValidationError as e:
    print(e)


Custom validation can be done on the model-level or individual attributes. Some things to note:
- all these validation methods can be done with pydantic dataclasses
- the value passed to the validator has already been coerced to the correct type unless `@validator` is used with `pre=True`. There is not type checking on the value returned from the validator
- if an [attribute's validity depends on another attribute](https://pydantic-docs.helpmanual.io/usage/models/#field-ordering), the dependent attribute should be defined after

In [ ]:
class GPSData_custom_validators(pydantic.BaseModel):
    latitude: float
    longitude: float
    speed: float
    heading: float

    @pydantic.validator("latitude")
    def clamp_latitude(cls, latitude):
        # if value is outside [-90, 90], just clamp to be within limits
        return max(-90.0, min(latitude, 90.0))

    # @pydantic.validator("longitude", pre=True)
    # def wrap_longitude(cls, longitude):
    #     # if value is outside [-180, 180], wrap around
    #     return ((longitude + 180) % 360) - 180
    
    @pydantic.validator("longitude")
    def wrap_longitude(cls, longitude):
        # if value is outside [-180, 180], wrap around
        return ((longitude + 180) % 360) - 180
    
    # @pydantic.validator("speed")
    # def validate_speed(cls, speed):
    #     # just raise an error instead of fixing
    #     if not speed >= 0:
    #         raise ValueError("speed should be non-negative")
    #     return speed

    @pydantic.validator("speed")
    def clamp_speed(cls, speed):
        # if negative, clamp to 0
        return max(speed, 0.0)

    @pydantic.validator("heading")
    def wrap_heading(cls, heading):
        # wrap around to be between [0, 360)
        return heading % 360

class GPSData_custom_root_validators(pydantic.BaseModel):
    latitude: float
    longitude: float
    speed: float
    heading: float

    @pydantic.root_validator
    def clamp_values(cls, values):
        values["latitude"] = max(-90.0, min(values["latitude"], 90.0))
        return values

    @pydantic.root_validator
    def handle_negative_speed(cls, values):
        if values["speed"] < 0:
            values["speed"] *= -1
            values["heading"] += 180
        return values

    @pydantic.root_validator
    def wrap_values(cls, values):
        values["longitude"] = ((values["longitude"] + 180) % 360) - 180
        values["heading"] = values["heading"] % 360
        return values

In [ ]:
print(
    GPSData_pydantic_base_model.parse_obj(gps_data_out_constraint),
    GPSData_custom_validators.parse_obj(gps_data_out_constraint),
    GPSData_custom_root_validators.parse_obj(gps_data_out_constraint),
    sep="\n"
)

# BaseSettings
The BaseSettings class works similarly to BaseModel but when instantiating, but it tries to get any missing values using case insensitive environment variables, and variables in a `.env` or secrets file.

It also supports custom sources that we could create to query the asset library, for example. So for something like the agency_id it might look like this:

### ToDo: investigate BaseSettings

In [ ]:
from pydantic import BaseSettings

def query_cdf(settings: BaseSettings):
  if should_query_cdf:
    response = response_from_asset_library()
    return response.json()

class Settings(BaseSettings):
  agency_id: str = "no_agency"
  class Config:
      @classmethod
      def customise_sources(cls, init, env, file):
        return (init, env, query_cdf, file)

# parser = ArgumentParser()
# parser.add_argument("--agency-id")
# arg_dict = vars(parser.parse_args())
arg_dict = {"agency_id": "example_agency_id"}

config = Settings(_env_file=".env", **arg_dict)

In [ ]:
from datetime import date
import os
from pydantic import BaseModel, root_validator, validator, PositiveFloat, validate_arguments, HttpUrl, FilePath
from pydantic.typing import Literal, Optional

class GTFSRealtimeConfig_pydantic(BaseModel):
    vehicle_positions_url: Optional[str]
    trip_updates_url: Optional[str]
    alerts_url: Optional[str]
    vehicle_id_field: Literal["id", "label"] = "id"
    max_polling_rate: PositiveFloat = 15
    subscribed_till: Optional[date]

    @root_validator(pre=True)
    def get_gtfs_realtime_api_from_features(cls, values):
        if "Features" in values:
            values = values["Features"]
        if "gtfs-realtime" in values:
            values = values["gtfs-realtime"]
        return values

    @root_validator(pre=True)
    def disregard_empty_values(cls, values):
        return {k: v for k, v in values.items() if v}

    @root_validator()
    def ensure_at_least_one_url(cls, values):
        if not any(
            values.get(key)
            for key in ("vehicle_positions_url", "trip_updates_url", "alerts_url")
        ):
            raise ValueError("At least one api endpoint is required")
        return values

    @classmethod
    @validate_arguments
    def from_inputs(
        cls,
        feature_persistence_url: HttpUrl = None,
        agency_id: str = None,
        should_query_feature_api: bool = False,
        config_file: FilePath = None,
    ):
        # default config file location
        config_dir = os.path.join(os.path.dirname(os.getcwd()), "tsp_gtfs_realtime", "config")
        default_config_file = os.path.join(config_dir, "gtfs-realtime-api-poller.json")

        # query feature persistence api
        if should_query_feature_api:
            if config_file:
                print(
                    f"config_file provided, but {should_query_feature_api=}, disregarding config_file"
                )
            if not (feature_persistence_url and agency_id):
                raise ValueError(
                    "feature_persistence_url and agency_id are required to query feature api"
                )
            url = f"{feature_persistence_url}/FeaturePersistence?AgencyGUID={agency_id}"
            response = requests.get(url)
            return GTFSRealtimeConfig_pydantic.parse_raw(response.content)
        # else create from config_file
        return GTFSRealtimeConfig_pydantic.parse_file(
            config_file or default_config_file
        )


In [ ]:
feature_json = '{"gtfs-realtime": {"vehicle_positions_url":"http://uctransit.info/gtfs-rt/vehiclepositions","trip_updates_url":"http://uctransit.info/gtfs-rt/tripupdates","alerts_url":"http://uctransit.info/gtfs-rt/alerts","vehicle_id_field":"label","max_polling_rate":"","subscribed_till":""}}'
feature_dict = {
    "gtfs-realtime": {
        "vehicle_positions_url": "http://uctransit.info/gtfs-rt/vehiclepositions",
        "trip_updates_url": "http://uctransit.info/gtfs-rt/tripupdates",
        "alerts_url": "http://uctransit.info/gtfs-rt/alerts",
        "vehicle_id_field": "label",
        "max_polling_rate": "",
        "subscribed_till": "",
    }
}
feature = feature_dict.get("gtfs-realtime")

GTFSRealtimeConfig_pydantic.parse_obj(
    {
        "gtfs-realtime": {
            "vehicle_positions_url": "asdfasdf",
            "trip_updates_url": "",
            "alerts_url": "",
            "vehicle_id_field": "label",
            "max_polling_rate": "",
            "subscribed_till": "",
        }
    }
)

In [ ]:
GTFSRealtimeConfig_pydantic.parse_file("../config/gtfs-realtime-api-poller.json")

In [ ]:
import requests

feature_persistence_url = "https://2g4ct7tedk.execute-api.us-east-1.amazonaws.com/default"
agency_id = "unioncity"

response = requests.get(f"{feature_persistence_url}/FeaturePersistence?AgencyGUID={agency_id}")
GTFSRealtimeConfig_pydantic.parse_raw(response.content)

In [ ]:
feature_persistence_url = "https://2g4ct7tedk.execute-api.us-east-1.amazonaws.com/default"
agency_id = "unioncity"

cfg_from_api = GTFSRealtimeConfig_pydantic.from_inputs(
    feature_persistence_url=feature_persistence_url,
    agency_id=agency_id,
    should_query_feature_api=True,
)

cfg_from_file = GTFSRealtimeConfig_pydantic.from_inputs(
    config_file="../tsp_gtfs_realtime/config/gtfs-realtime-api-poller.json"
)

cfg_from_default_file = GTFSRealtimeConfig_pydantic.from_inputs()

In [ ]:
from tsp_gtfs_realtime import GTFSRealtimeConfig

GTFSRealtimeConfig()

In [ ]:
import json

integration_com_raw = """{"attributes":{"gttSerial":"643","serial":"643","addressMAC":"00:00:00:d0:06:43","preemptionLicense":"pending","integration":"gtfs-realtime","uniqueId":"643dde30-1de5-47c0-ac61-5580a7232e9f"},"groups":{"ownedby":["/unioncity/unioncity"]},"devices":{"installedat":["union-city-vehicle-643"]},"category":"device","templateId":"integrationcom","description":"TSP in the Cloud Demo","state":"active","deviceId":"tsp-gtfs-realtime-643"}"""
integration_com_dict = json.loads(integration_com_raw)

### IntegrationCom

template
``` yaml
templateId: integrationcom
category: device
properties:
  description:
    type: [string,null]
  serial:
    type: string
  gttSerial:
    type: string
  addressMAC:
    type: [string,null]
  uniqueId:
    type: string
  preemptionLicense:
    type: [string,null]
    enum: [pending,inactive,active,decommissioned,transferred]
  integration:
    type: [string,null]
    enum: [Whelen,Teletrac,gtfs-realtime,Misc.]
relations:
  out:
    ownedby: [agency]
required:
  - uniqueId
  - addressMAC
  - serial
```

example response:
``` yaml
attributes:
  gttSerial: '643'
  serial: '643'
  addressMAC: 00:00:00:d0:06:43
  preemptionLicense: pending
  integration: gtfs-realtime
  uniqueId: 643dde30-1de5-47c0-ac61-5580a7232e9f
groups:
  ownedby:
  - "/unioncity/unioncity"
devices:
  installedat:
  - union-city-vehicle-643
category: device
templateId: integrationcom
description: TSP in the Cloud Demo
state: active
deviceId: tsp-gtfs-realtime-643
```

In [ ]:
from pydantic import BaseModel, conint, constr, Field
from pydantic.typing import Literal, List

class IntegrationComAttributes(BaseModel):
    serial: str
    gtt_serial: str = Field(alias="gttSerial")
    address_mac: constr(regex=r"^([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2})$") = Field(alias="addressMAC")
    unique_id: str = Field(alias="uniqueId")
    license: Literal["pending", "inactive", "active", "decommissioned", "transferred"] = Field(alias="preemptionLicense")
    integration: Literal["Whelen", "Teletrac", "gtfs-realtime", "Misc."]

class IntegrationComOwnedBy(BaseModel):
    # match "/region/agency", where each is alphanumeric with underscores/hyphens
    owned_by: List[constr(regex=r"^\/[\w-]+\/[\w-]+$")] = Field(alias="ownedby")

class IntegrationComGroups(BaseModel):
    # match "/region/agency", where each is alphanumeric with underscores/hyphens
    dir_out: IntegrationComOwnedBy = Field(alias="out")

class IntegrationComInstalledAt(BaseModel):
    installed_at: List[str] = Field(alias="installedat")

class IntegrationComDevices(BaseModel):
    dir_in: IntegrationComInstalledAt = Field(alias="in")

class IntegrationCom(BaseModel):
    description: str = None
    attributes: IntegrationComAttributes
    groups: IntegrationComGroups
    devices: IntegrationComDevices
    category: Literal["device"]
    template_id: Literal["integrationcom"] = Field(alias="templateId")
    description: str = None
    state: Literal["active"]
    device_id: str = Field(alias="deviceId")

In [ ]:
device = IntegrationCom.parse_raw(integration_com_raw)
# device = IntegrationCom.parse_obj(integration_com_dict)
device

In [ ]:
integration_com_dict
# integration_com_dict["attributes"].get("addressMAC")

In [ ]:
import boto3
from botocore.awsrequest import AWSRequest
from botocore.endpoint import URLLib3Session
from botocore.auth import SigV4Auth
from botocore.compat import urlencode, quote


base_url = "https://oo9fn5p38b.execute-api.us-east-1.amazonaws.com/Prod"
region_name = "UnionCity" # not case sensitive
agency_name = "UnionCity" # not case sensitive
device_id = "tsp-gtfs-realtime-643"
ancestor_path = quote(f"/{region_name}/{agency_name}", safe="")
print(f"{ancestor_path=}")

template_type_dict = {
    "type": "vehicleV2", # not case sensitive
    "state": "active",
}
template_type = "type=vehicleV2"

agency_devices_url = f"{base_url}/groups/{ancestor_path}/members/devices"
device_url = f"{base_url}/devices/{device_id}"
search_url_0 = f"{base_url}/search?{template_type}"
search_url_1 = f"{base_url}/search?{urlencode(template_type_dict)}"

print(f"{agency_devices_url=}")
print(f"{device_url=}")
print(f"{search_url_0=}")
print(f"{search_url_1=}")

headers = {
    "Accept": "application/vnd.aws-cdf-v2.0+json",
    "Content-Type": "application/vnd.aws-cdf-v2.0+json",
}

request_0 = AWSRequest(method="GET", url=search_url_0, headers=headers)
SigV4Auth(boto3.Session().get_credentials(), "execute-api","us-east-1").add_auth(
    request_0
)
response_0 = URLLib3Session().send(request_0.prepare()).content

request_1 = AWSRequest(method="GET", url=search_url_1, headers=headers)
SigV4Auth(boto3.Session().get_credentials(), "execute-api","us-east-1").add_auth(
    request_1
)
response_1 = URLLib3Session().send(request_1.prepare()).content

In [ ]:
import requests
from requests_aws_sign import AWSV4Sign
import boto3
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest
from botocore.compat import quote, urlencode
from botocore.endpoint import URLLib3Session

base_url = "https://oo9fn5p38b.execute-api.us-east-1.amazonaws.com/Prod"
region_name = "UnionCity" # not case sensitive
agency_name = "UnionCity" # not case sensitive
device_id = "tsp-gtfs-realtime-643"
ancestor_path = quote(f"/{region_name}/{agency_name}", safe="")

service = "execute-api"
credentials = boto3.Session().get_credentials()
aws_region = os.environ["AWS_REGION"]
auth = AWSV4Sign(credentials, aws_region, service)

headers = {
    "Accept": "application/vnd.aws-cdf-v2.0+json",
    "Content-Type": "application/vnd.aws-cdf-v2.0+json",
}

device_url = f"{base_url}/devices/{device_id}"
request_0 = AWSRequest(method="GET", url=device_url, headers=headers)
SigV4Auth(boto3.Session().get_credentials(), "execute-api","us-east-1").add_auth(
    request_0
)
response_0 = URLLib3Session().send(request_0.prepare()).content

auth = AWSV4Sign(credentials, aws_region, service)
request_1 = requests.get(device_url, headers=headers, auth=auth)
response_1 = request_1.content

In [ ]:
print(response_0)
print(response_1)
response_0 == response_1

In [ ]:
r0 = json.loads(response_0, strict=False)
print("r0", set(e["templateId"] for e in r0["results"]))

r1 = json.loads(response_1, strict=False)
print("r1", set(e["templateId"] for e in r1["results"]))

In [ ]:
class Agency:
    pass

class Region:
    pass

class Vehicle:
    pass

In [ ]:
import abc

from botocore.awsrequest import AWSRequest
from botocore.endpoint import URLLib3Session
from botocore.auth import SigV4Auth
from botocore.compat import urlencode

from pydantic import BaseModel, HttpUrl

class AssetLibraryBaseModel(BaseModel, abc.ABC):

    class Config:
        underscore_attrs_are_private = True

    _base_url = "https://oo9fn5p38b.execute-api.us-east-1.amazonaws.com/Prod"
    # _headers = {
    #     "Accept": "application/vnd.aws-cdf-v2.0+json",
    #     "Content-Type": "application/vnd.aws-cdf-v2.0+json",
    # }
    # _credentials = boto3.Session().get_credentials()

    @abc.abstractproperty
    def _url(self):
        pass

    @classmethod
    def _send_request(cls, url=None, region_name=None, params=None, data=None, headers=None) -> bytes:
        credentials = boto3.Session().get_credentials()
        url = url
        headers = {
            "Accept": "application/vnd.aws-cdf-v2.0+json",
            "Content-Type": "application/vnd.aws-cdf-v2.0+json",
        }
        method = "GET"
        region_name = region_name or "us-east-1"

        # AWSRequest is unable to handle params with identical keys,
        # so manually append to the url
        encoded_url = f"{url}?{urlencode(params)}" if params else url

        # the search API seems to be broken to some degree as only strings without
        # encoded characters work, and only with the \`eq\` query parameter

        request = AWSRequest(method=method, url=encoded_url, headers=headers, data=data)
        SigV4Auth(credentials, "execute-api", region_name).add_auth(request)
        response = URLLib3Session().send(request.prepare())
        if response.status_code >= 400 or not response.content:
            #! logging.debug(f"Invalid Asset Library response. url={response.url}, status_code={response.status_code}, headers={response.headers}, content={response.content}")
            raise ValueError(
                f"Invalid Asset Library response. {response.status_code=}, {response.content=}"
            )
        return response.content

    @classmethod
    @abc.abstractmethod
    def from_asset_library(cls):
        pass

In [ ]:
from pydantic import BaseModel, conint, constr, Field, root_validator, validate_arguments
from typing import Literal, List, Union, Dict
import requests

class IntegrationCom(AssetLibraryBaseModel):
    description: str = None
    category: Literal["device"]
    template_id: Literal["integrationcom"] = Field(alias="templateId")
    state: Literal["active"]
    device_id: str = Field(alias="deviceId")
    # attributes
    serial: str
    gtt_serial: str = Field(alias="gttSerial")
    address_mac: constr(regex=r"^([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2})$") = Field(alias="addressMAC")
    unique_id: str = Field(alias="uniqueId")
    license: Literal["pending", "inactive", "active", "decommissioned", "transferred"] = Field(alias="preemptionLicense")
    integration: Literal["Whelen", "Teletrac", "gtfs-realtime", "Misc."]
    # groups
    region_id: str
    agency_id: str
    # devices
    vehicle_id: str

    # used to lazy load related entities
    _region: Region = None
    _agency: Agency = None
    _vehicle: Vehicle = None

    @property
    def _url(self):
        return f"{self._base_url}/devices/{self.device_id}"

    @classmethod
    @validate_arguments
    def from_asset_library(cls,
        base_url: HttpUrl = None,
        device_id: str = None,
        filter: Dict[str, str] = None
    ):
        """generate IntegrationCom object by querying asset library

        Args:
            device_id (str):
                asset library deviceId to directly access the device
            filter (dict):
                dictionary of attributes to use search api. [more on filter params](https://github.com/aws/aws-connected-device-framework/blob/main/source/packages/services/assetlibrary/docs/swagger.yml#L195)
        
        Note that the search api accepts other comparisons than equals, but this
        function forces equality to be used, and only works with strings. This could
        be expanded if needed.

        Returns:
            IntegrationCom: instantiated class object
        """
        # ToDo: get base_url from environment variable
        base_url = base_url or "https://oo9fn5p38b.execute-api.us-east-1.amazonaws.com/Prod"

        if filter:
            if device_id:
                #! logging.warning("device_id and filter both supplied, trying first with device_id")
                try:
                    return cls.from_asset_library(base_url=base_url, device_id=device_id)
                except ValueError:
                    #! logging.warning("retrying using filter")
                    pass
            # convert to dictionary of attribute fields/values to 2-tuple
            # note that the keys and format of filter is not checked
            params = [
                ("type", "integrationcom"),
                *[("eq", f"{k}:{v}") for k, v in filter.items()],
            ]
            #! logging.debug(f"attempting to query using search: {filter}")
            # manually parse and re-query since groups/devices
            # are not included when using the search api
            search_url = f"{base_url}/search"
            search_dict = json.loads(
                cls._send_request(url=search_url, params=params),
                strict=False,
            )

            count = search_dict["pagination"]["count"]
            if count != 1:
                raise ValueError(f"provided filter returned {'no' if count == 0 else 'multiple'} results")
            # use the discovered device_id to re-query
            device_id = search_dict["results"][0]["deviceId"]
            #! logging.info(f"found device_id as {device_id}. re-querying to get device")
            return cls.from_asset_library(base_url=base_url, device_id=device_id)

        # else, query using device_id
        #! logging.debug(f"attempting to query using device_id: {device_url}")
        device_url = f"{base_url}/devices/{device_id}"

        return cls.parse_raw(cls._send_request(url=device_url))

    @property
    def region(self) -> Region:
        self._region = self._region or Region.from_asset_library(
            group_path=f"/{self.region_id}",
        )
        return self._region

    @property
    def agency(self) -> Agency:
        self._agency = self._agency or Agency.from_asset_library(
            group_path=f"/{self.region_id}/{self.agency_id}",
        )
        return self._agency

    @property
    def vehicle(self) -> Vehicle:
        self._vehicle = self._vehicle or Vehicle.from_asset_library(
            device_id=self.vehicle_id,
        )
        return self._vehicle

    @root_validator(pre=True)
    def validate_and_flatten_integration_com(cls, values):
        # warn of empty response
        #! logging.debug(f"{values=}")
        # get the path to the agency group to parse region/agency
        # ToDo: log/handle parsing errors, empty/multiple values, etc.
        #! when using postman, there is no intermediate field for out/in, handle both
        owned_by = (
            values["groups"].get("out") or values["groups"]
        ).get("ownedby")[0]

        installed_at = (
            values["devices"].get("in") or values["devices"]
        ).get("installedat")[0]

        values.update(
            {
                "region_id": owned_by.split("/")[1],
                "agency_id": owned_by.split("/")[2],
                "vehicle_id": installed_at,
            }
        )
        # flatten attributes
        values.update(values["attributes"])

        return values

In [ ]:
ic0 = IntegrationCom.from_asset_library(device_id="tsp-gtfs-realtime-643")
ic1 = IntegrationCom.from_asset_library(device_id="tsp-gtfs-realtime-543", filter={"serial": 643})

print(ic0)
print(f"{(ic0 == ic1)=}")

In [ ]:
from pydantic import BaseModel, conint, constr, Field, root_validator, validate_arguments
from typing import Literal, List, Union, Dict
import requests

class Vehicle(AssetLibraryBaseModel):
    description: str = None
    category: Literal["device"]
    template_id: Literal["vehiclev2"] = Field(alias="templateId")
    state: Literal["active"]
    device_id: str = Field(alias="deviceId")
    # attributes
    VID: conint(ge=1, le=9999)
    name: str = None
    priority: Literal["High", "Low"]
    type_: str = Field(alias="type", default=None)
    class_: str = Field(alias="class")
    unique_id: str = Field(alias="uniqueId")
    # groups
    region_id: str
    agency_id: str
    # devices
    installed_device_ids: List[str]

    # used to lazy load related entities
    _region: Region = None
    _agency: Agency = None
    _installed_devices: List[Union[IntegrationCom, str]] = None
    _integration_coms: List[IntegrationCom] = None

    @property
    def _url(self):
        return f"{self._base_url}/devices/{self.device_id}"

    @classmethod
    @validate_arguments
    def from_asset_library(cls,
        base_url: HttpUrl = None,
        device_id: str = None,
        filter: Dict[str, str] = None
    ):
        """generate Vehicle object by querying asset library

        Args:
            device_id (str):
                asset library deviceId to directly access the device
            filter (dict):
                dictionary of attributes to use search api. [more on filter params](https://github.com/aws/aws-connected-device-framework/blob/main/source/packages/services/assetlibrary/docs/swagger.yml#L195)
        
        Note that the search api accepts other comparisons than equals, but this
        function forces equality to be used, and only works with strings. This could
        be expanded if needed.

        Returns:
            Vehicle: instantiated class object
        """
        # ToDo: get base_url from environment variable
        base_url = base_url or "https://oo9fn5p38b.execute-api.us-east-1.amazonaws.com/Prod"

        if filter:
            if device_id:
                #! logging.warning("device_id and filter both supplied, trying first with device_id")
                try:
                    return cls.from_asset_library(base_url=base_url, device_id=device_id)
                except ValueError:
                    #! logging.warning("retrying using filter")
                    pass
            # convert to dictionary of attribute fields/values to 2-tuple
            # note that the keys and format of filter is not checked
            params = [
                ("type", "vehiclev2"),
                *[("eq", f"{k}:{v}") for k, v in filter.items()],
            ]
            
            #! logging.debug(f"attempting to query using search: {filter}")
            # manually parse and re-query since groups/devices
            # are not included when using the search api
            search_url = f"{base_url}/search"
            search_dict = json.loads(
                cls._send_request(url=search_url, params=params),
                strict=False,
            )

            count = search_dict["pagination"]["count"]
            if count != 1:
                raise ValueError(f"provided filter returned {'no' if count == 0 else 'multiple'} results")
            # use the discovered device_id to re-query
            device_id = search_dict["results"][0]["deviceId"]
            #! logging.info(f"found device_id as {device_id}. re-querying to get device")
            return cls.from_asset_library(base_url=base_url, device_id=device_id)

        # else, query using device_id
        #! logging.debug(f"attempting to query using device_id: {device_url}")
        device_url = f"{base_url}/devices/{device_id}"

        return cls.parse_raw(cls._send_request(url=device_url))

    @property
    def region(self) -> Region:
        self._region = self._region or Region.from_asset_library(
            group_path=f"/{self.region_id}",
        )
        return self._region

    @property
    def agency(self) -> Agency:
        self._agency = self._agency or Agency.from_asset_library(
            group_path=f"/{self.region_id}/{self.agency_id}",
        )
        return self._agency

    @property
    def installed_devices(self) -> List[Union[IntegrationCom, str]]:
        # note that this stores the actual IntegrationCom entities, but only the
        # device_id for other devices. this could be expanded once other models exist
        def integration_com_or_device_id(device_id: str) -> Union[IntegrationCom, str]:
            try:
                return IntegrationCom.from_asset_library(device_id=device_id)
            except ValueError as e:
                #! logging.debug(e)
                return device_id

        self._installed_devices = self._installed_devices or [
            integration_com_or_device_id(d) for d in self.installed_device_ids
        ]
        return self._installed_devices

    @property
    def integration_com(self) -> IntegrationCom:
        # get installed IntegrationCom device, assert singular entity
        self._integration_coms = self._integration_coms or [
            d for d in self.installed_devices if isinstance(d, IntegrationCom)
        ]
        assert len(self._integration_coms) == 1
        return self._integration_coms[0]

    @root_validator(pre=True)
    def validate_and_flatten_vehicle(cls, values):
        # warn of empty response
        #! logging.debug(f"{values=}")
        # get the path to the agency group to parse region/agency
        # ToDo: log/handle parsing errors, empty/multiple values, etc.
        #! when using postman, there is no intermediate field for out/in, handle both
        owned_by = (
            values["groups"].get("out") or values["groups"]
        ).get("ownedby")[0]

        installed_at = (
            values["devices"].get("out") or values["devices"]
        ).get("installedat")

        values.update(
            {
                "region_id": owned_by.split("/")[1],
                "agency_id": owned_by.split("/")[2],
                "installed_device_ids": installed_at,
            }
        )
        # flatten attributes
        values.update(values["attributes"])

        return values

In [ ]:
v0 = Vehicle.from_asset_library(device_id="union-city-vehicle-643")
v1 = Vehicle.from_asset_library(device_id="union-city-vehicle-543", filter={"name": "vehicle-643"})

#! It seems like it is not possible to search using VID because "eq" requires the field
#! to be a string type, but VID is a number. Using gte, lte, etc. timed out after 30s

print(v0)
print(f"{(v0 == v1)=}")

In [ ]:
from pydantic import BaseModel, conint, constr, Field, root_validator, validate_arguments
from typing import Literal, List, Union, Dict
import requests

class Agency(AssetLibraryBaseModel):
    description: str = None
    category: Literal["group"]
    template_id: Literal["agency"] = Field(alias="templateId")
    name: str
    group_path: str = Field(alias="groupPath")
    parent_path: str = Field(alias="parentPath")
    # attributes
    city: str
    state: str
    timezone: Literal["Central", "Mountain", "Eastern", "Pacific", "Arizona"]
    agency_code: conint(ge=1, le=254) = Field(alias="agencyCode")
    unique_id: str = Field(alias="agencyID") # note name change
    vps_cert_id: str = Field(alias="vpsCertId")
    cert2100_id: str = Field(alias="Cert2100Id", default=None)
    priority: Literal["High", "Low"]
    cms_id: str = Field(alias="CMSId", default=None)
    ca_cert_id: str = Field(alias="caCertId")
    display_name: str = Field(alias="displayName", default=None)
    # groups
    region_id: str
    # member devices requires querying

    # used to lazy load related entities
    _region: Region = None
    _devices: List[Union[Vehicle, IntegrationCom, str]] = None
    _integration_coms: List[IntegrationCom] = None
    _vehicles: List[Vehicle] = None

    @property
    def _url(self):
        return f"{self._base_url}/groups/{quote(self.group_path, safe='')}"

    @classmethod
    @validate_arguments
    def from_asset_library(cls,
        base_url: HttpUrl = None,
        group_path: str = None,
        filter: Dict[str, str] = None
    ):
        """generate Agency object by querying asset library

        Args:
            group_path (str):
                asset library groupPath to directly access the group
            filter (dict):
                dictionary of attributes to use search api. [more on filter params](https://github.com/aws/aws-connected-device-framework/blob/main/source/packages/services/assetlibrary/docs/swagger.yml#L195)
        
        Note that the search api accepts other comparisons than equals, but this
        function forces equality to be used, and only works with strings. This could
        be expanded if needed.

        Returns:
            Agency: instantiated class object
        """
        # ToDo: get base_url from environment variable
        base_url = base_url or "https://oo9fn5p38b.execute-api.us-east-1.amazonaws.com/Prod"
        # urlencode the group_path to handle /
        group_path = quote(group_path, safe="%") if group_path else None

        if filter:
            if group_path:
                #! logging.warning("group_path and filter both supplied, trying first with group_path")
                try:
                    return cls.from_asset_library(base_url=base_url, group_path=group_path)
                except ValueError:
                    #! logging.warning("retrying using filter")
                    pass
            # convert to dictionary of attribute fields/values to 2-tuple
            # note that the keys and format of filter is not checked
            params = [
                ("type", "agency"),
                *[("eq", f"{k}:{v}") for k, v in filter.items()],
            ]
            #! logging.debug(f"attempting to query using search: {filter}")
            # manually parse and re-query since groups/devices
            # are not included when using the search api
            search_url = f"{base_url}/search"
            search_dict = json.loads(
                cls._send_request(url=search_url, params=params),
                strict=False,
            )

            count = search_dict["pagination"]["count"]
            if count != 1:
                raise ValueError(f"provided filter returned {'no' if count == 0 else 'multiple'} results")
            # use the discovered group_path to re-query
            group_path = search_dict["results"][0]["groupPath"]
            #! logging.info(f"found group_path as {group_path}. re-querying to get group")
            return cls.from_asset_library(base_url=base_url, group_path=group_path)

        # else, query using group_path
        #! logging.debug(f"attempting to query using group_path: {group_url}")
        group_url = f"{base_url}/groups/{group_path}"

        return cls.parse_raw(cls._send_request(url=group_url))

    @property
    def region(self) -> Region:
        self._region = self._region or Region.from_asset_library(
            group_path=self.parent_path,
        )
        return self._region

    @property
    def devices(self) -> List[Union[Vehicle, IntegrationCom, str]]:
        # note that this stores the actual Vehicle/IntegrationCom entities, but only the
        # device_id for other devices. this could be expanded once other models exist
        if not self._devices:
            members_url = f"{self._url}/members/devices"
            members = json.loads(
                self._send_request(url=members_url),
                strict=False,
            )
            self._devices = [
                Vehicle.from_asset_library(device_id=d["deviceId"])
                if d["templateId"].lower() == "vehiclev2"
                else IntegrationCom.from_asset_library(device_id=d["deviceId"])
                if d["templateId"].lower() == "integrationcom"
                else d["deviceId"]
                for d in members["results"]
            ]
        return self._devices

    @property
    def vehicles(self) -> List[Vehicle]:
        # get Vehicle devices from member devices
        self._vehicles = self._vehicles or [
            d for d in self.devices if isinstance(d, Vehicle)
        ]
        return self._vehicles

    @property
    def integration_coms(self) -> List[IntegrationCom]:
        # get IntegrationCom devices from member devices
        self._integration_coms = self._integration_coms or [
            d for d in self.devices if isinstance(d, IntegrationCom)
        ]
        return self._integration_coms

    @root_validator(pre=True)
    def validate_and_flatten_agency(cls, values):
        # warn of empty response
        #! logging.debug(f"{values=}")
        # get parentPath to parse region
        values["region_id"] = values["parentPath"].split("/")[-1]

        # flatten attributes
        values.update(values["attributes"])

        return values

In [ ]:
# case insensitive
region_name = "UnionCity"
agency_name = "uNiOncITy"
group_path = f"/{region_name}/{agency_name}"
group_path_encoded = quote(group_path, safe="")

a0 = Agency.from_asset_library(group_path=group_path)
a1 = Agency.from_asset_library(group_path=group_path_encoded)
a2 = Agency.from_asset_library(filter={"displayName": "UnionCity"})

print(a0)
print(f"{(a0 == a1 == a2)=}")

In [ ]:
from pydantic import BaseModel, conint, constr, Field, root_validator, validate_arguments
from typing import Literal, List, Union, Dict
import requests

class Region(AssetLibraryBaseModel):
    description: str = None
    category: Literal["group"]
    template_id: Literal["region"] = Field(alias="templateId")
    name: str
    group_path: str = Field(alias="groupPath")
    parent_path: Literal["/"] = Field(alias="parentPath")
    # attributes
    ca_cert_id: str = Field(alias="caCertId")
    unique_id: str = Field(alias="regionGUID") # note name change
    display_name: str = Field(alias="displayName", default=None)
    # child groups requires querying

    # used to lazy load related entities
    _agencies: List[Agency] = None
    # region has no child devices

    @property
    def _url(self):
        return f"{self._base_url}/groups/{quote(self.group_path, safe='')}"

    @classmethod
    @validate_arguments
    def from_asset_library(cls,
        base_url: HttpUrl = None,
        group_path: str = None,
        filter: Dict[str, str] = None
    ):
        """generate Region object by querying asset library

        Args:
            group_path (str):
                asset library groupPath to directly access the group
            filter (dict):
                dictionary of attributes to use search api. [more on filter params](https://github.com/aws/aws-connected-device-framework/blob/main/source/packages/services/assetlibrary/docs/swagger.yml#L195)
        
        Note that the search api accepts other comparisons than equals, but this
        function forces equality to be used, and only works with strings. This could
        be expanded if needed.

        Returns:
            Region: instantiated class object
        """
        # ToDo: get base_url from environment variable
        base_url = base_url or "https://oo9fn5p38b.execute-api.us-east-1.amazonaws.com/Prod"
        # urlencode the group_path to handle /
        group_path = quote(group_path, safe="%") if group_path else None

        if filter:
            if group_path:
                #! logging.warning("group_path and filter both supplied, trying first with group_path")
                try:
                    return cls.from_asset_library(base_url=base_url, group_path=group_path)
                except ValueError:
                    #! logging.warning("retrying using filter")
                    pass
            # convert to dictionary of attribute fields/values to 2-tuple
            # note that the keys and format of filter is not checked
            params = [
                ("type", "region"),
                *[("eq", f"{k}:{v}") for k, v in filter.items()],
            ]
            #! logging.debug(f"attempting to query using search: {filter}")
            # manually parse and re-query since groups/devices
            # are not included when using the search api
            search_url = f"{base_url}/search"
            search_dict = json.loads(
                cls._send_request(url=search_url, params=params),
                strict=False,
            )

            count = search_dict["pagination"]["count"]
            if count != 1:
                raise ValueError(f"provided filter returned {'no' if count == 0 else 'multiple'} results")
            # use the discovered group_path to re-query
            group_path = search_dict["results"][0]["groupPath"]
            #! logging.info(f"found group_path as {group_path}. re-querying to get group")
            return cls.from_asset_library(base_url=base_url, group_path=group_path)

        # else, query using group_path
        #! logging.debug(f"attempting to query using group_path: {group_url}")
        group_url = f"{base_url}/groups/{group_path}"

        return cls.parse_raw(cls._send_request(url=group_url))

    @property
    def agencies(self) -> List[Agency]:
        if not self._agencies:
            members_url = f"{self._url}/members/groups"
            members = json.loads(
                self._send_request(url=members_url),
                strict=False,
            )
            self._agencies = [
                Agency.from_asset_library(group_path=group["groupPath"])
                for group in members["results"]
                if group["templateId"].lower() == "agency"
            ]
        return self._agencies

    @root_validator(pre=True)
    def validate_and_flatten_agency(cls, values):
        # warn of empty response
        #! logging.debug(f"{values=}")
        # flatten attributes
        values.update(values["attributes"])

        return values

In [ ]:
# case insensitive
region_name = "unioncity"
group_path = f"/{region_name}"
group_path_encoded = quote(group_path, safe="")

r0 = Region.from_asset_library(group_path=group_path)
r1 = Region.from_asset_library(group_path=group_path_encoded)
r2 = Region.from_asset_library(filter={"displayName": "UnionCity"})

print(r0)
print(f"{(r0 == r1 == r2)=}")

In [ ]:
print(f"{(ic0.vehicle == v0)=}")
print(f"{(v0.integration_com == ic0)=}")

In [ ]:
print(f"{(ic0.agency == v0.agency == a0 == r0.agencies[0])=}")
print(f"{(ic0.region == v0.region == a0.region == r0)=}")

In [ ]:
print(f"{(ic0 in a0.integration_coms)=}")
print(f"{(v0 in a0.vehicles)=}")
print(f"{all(d in a0.devices for d in [ic0, ic1, v0, v1])=}")
print(f"{all(d in r0.agencies[0].devices for d in [ic0, ic1, v0, v1])=}")

In [ ]:
base_url = "https://oo9fn5p38b.execute-api.us-east-1.amazonaws.com/Prod"

print(f"{base_url}/groups/{quote(r0.group_path, safe='')}/members/groups")
print(f"{r0._url}/members/groups")
print(r0._url)
print(r0._base_url)

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, ClassVar, ForwardRef
import abc

class GroupBaseModel(BaseModel):
    description: str = None
    category: Literal["group"]
    # template_id: ClassVar[str] = "test"
    # template_id = "test"
    # template_id: Literal["region", "agency"] = Field(alias="templateId")
    _template_id: str = Field(alias="templateId", const=True, default=ForwardRef("template_id"))
    name: str
    group_path: str = Field(alias="groupPath")
    parent_path: Literal["/"] = Field(alias="parentPath")

    @property
    def _url(self):
        return f"{self._base_url}/groups/{quote(self.group_path, safe='')}"

    @classmethod
    @property
    @abc.abstractmethod
    def template_id(self) -> str:
        raise NotImplementedError
    

class Region(GroupBaseModel):
    template_id: ClassVar[str] = "region"

In [ ]:
Region.__name__

In [ ]:
from tsp_gtfs_realtime.core.data_model import Region

ic_device_id = "tsp-gtfs-realtime-643"
invalid_ic_device_id = "tsp-gtfs-realtime-543"
ic_serial_search_attributes = {"serial": 643}

vehicle_device_id = "union-city-vehicle-643"
invalid_vehicle_device_id = "union-city-vehicle-543"
vehicle_name_search_attributes = {"name": "vehicle-643"}

# case insensitive
region_name = "UnionCity"
agency_name = "uNiOncITy"
region_path = f"/{region_name}"
agency_path = f"/{region_name}/{agency_name}"
display_name_search_attributes = {"displayName": "UnionCity"}

region = Region.from_asset_library(group_path=region_path)
region1 = Region.from_asset_library(group_path=agency_path)


In [ ]:
mac_address = "00:00:00:d0:06:43"
mac_address = "FF:FF:FF:7F:FF:FF"
mac_address_1 = mac_address.replace(":", "")[-6:]
mac_address_2 = bin(int(mac_address_1, 16))[2:].zfill(24)

orrer = f"{8388608:024b}"
mac_address_3 = ""

for i in range(0, 24):
    mac_address_3 += str(int(orrer[i]) | int(mac_address_2[i]))

mac_address_4 = int(mac_address_3, 2)
print(mac_address)
print(mac_address_1)
print(mac_address_2)
print(orrer)
print(mac_address_3)
print(mac_address_4)
print(hex(mac_address_4))

In [ ]:
int(mac_address_4).to_bytes(4, byteorder="little")
int(int(mac_address_4).to_bytes(4, byteorder="little")).to_bytes(4, byteorder="little")

In [ ]:
from typing import Literal
from pydantic import BaseModel, Field
class R(BaseModel):
    template_id: Literal["region"] = Field(alias="templateId")

In [ ]:
Region.template_id

In [ ]:

#! ToDo: continue
class DeviceBaseModel(AssetLibraryBaseModel):
    description: str = None
    category: Literal["group"]
    template_id: str = Field(alias="templateId")
    name: str
    group_path: str = Field(alias="groupPath")
    parent_path: str = Field(alias="parentPath")

    @classmethod
    @property
    @abc.abstractmethod
    def _template_id(self) -> str:
        raise NotImplementedError

    @validator("template_id")
    def validate_template_id(cls, value):
        if value != cls._template_id:
            raise ValueError(
                f"template_id does not match; expected='{cls._template_id}', given='{value}'"
            )
        return value

    @property
    def _url(self):
        return f"{self._base_url}/groups/{quote(self.group_path, safe='')}"

    @classmethod
    @validate_arguments
    def from_asset_library(
        cls,
        base_url: HttpUrl = None,
        group_path: str = None,
        search_attributes: Dict[str, str] = None,
    ):
        f"""generate {cls.__name__} object by querying asset library

        Args:
            group_path (str):
                asset library groupPath to directly access the group
            search_attributes (dict):
                dictionary of attributes to use search api. [more on filter params](https://github.com/aws/aws-connected-device-framework/blob/main/source/packages/services/assetlibrary/docs/swagger.yml#L195)

        Note that the search api accepts other comparisons than equals, but this
        function forces equality to be used, and only works with strings. This could
        be expanded if needed.

        Returns:
            {cls.__name__}: instantiated class object
        """
        # ToDo: get base_url from environment variable
        base_url = (
            base_url or "https://oo9fn5p38b.execute-api.us-east-1.amazonaws.com/Prod"
        )
        # urlencode the group_path to handle /
        group_path = quote(group_path, safe="%") if group_path else None

        if search_attributes:
            if group_path:
                logging.warning(
                    "group_path and search_attributes both supplied, trying first with group_path"
                )
                try:
                    return cls.from_asset_library(
                        base_url=base_url, group_path=group_path
                    )
                except ValueError:
                    logging.warning(
                        f"unable to query {group_path=} retrying using search_attributes"
                    )
            # convert to dictionary of attribute fields/values to 2-tuple
            # note that the keys and format of search_attributes is not checked
            params = [
                ("type", cls._template_id),
                *[("eq", f"{k}:{v}") for k, v in search_attributes.items()],
            ]
            logging.debug(f"attempting to query using search: {search_attributes}")
            # manually parse and re-query since groups/devices
            # are not included when using the search api
            search_url = f"{base_url}/search"
            search_dict = json.loads(
                cls._send_request(url=search_url, params=params),
                strict=False,
            )

            count = search_dict["pagination"]["count"]
            if count != 1:
                raise ValueError(
                    f"provided search_attributes returned {'no' if count == 0 else 'multiple'} results"
                )
            # use the discovered group_path to re-query
            group_path = search_dict["results"][0]["groupPath"]
            logging.info(f"found group_path as {group_path}. re-querying to get group")
            return cls.from_asset_library(base_url=base_url, group_path=group_path)

        # else, query using group_path
        group_url = f"{base_url}/groups/{group_path}"
        logging.debug(f"attempting to query using group_path: {group_url}")

        return cls.parse_raw(cls._send_request(url=group_url))